In [1]:
import pandas as pd
import numpy as np
import torch
from torch import optim, nn
from torch.utils.data import TensorDataset, DataLoader
from plotly import offline as py

import modules

### Load OHLCV and indicator data

In [12]:
data = pd.read_csv(
    'data/indicators.csv',
    index_col='sample_time',
    parse_dates=True,
    nrows=50001
).iloc[:-1]
ohlcv = data[[
#     'open_m1',
#     'high_m1',
#     'low_m1',
    'close_m1',
#     'volume_tick_m1'
]]
indicators = data[[
    'ema_close_12_m1',
    'ema_close_16_m1',
    'ema_close_26_m1',
    'ema_close_50_m1',
    'ema_close_60_m1',
    'ema_close_100_m1',
    'ema_close_200_m1',
    'ema_close_240_m1',
#     'ema_volume_tick_12_m1',
#     'ema_volume_tick_26_m1',
    'macd_close_12_26_m1',
#     'macd_sig_close_12_26_9_m1',
#     'macd_hist_close_12_26_9_m1',
    'macd_close_16_100_m1',
#     'macd_sig_close_16_100_2_m1',
#     'macd_hist_close_16_100_2_m1',
    'macd_close_60_240_m1',
#     'macd_sig_close_60_240_10_m1',
#     'macd_hist_close_60_240_10_m1',
#     'macd_volume_tick_12_26_m1',
#     'macd_sig_volume_tick_12_26_9_m1',
#     'macd_hist_volume_tick_12_26_9_m1',
]]

### Run EMA test

In [13]:
x = torch.tensor(ohlcv.values.astype(np.float32))
y = torch.tensor(indicators.values.astype(np.float32))

dataset = TensorDataset(x, y)
loader = DataLoader(dataset, batch_size=1024*64, num_workers=4)
loss_fn = nn.MSELoss()

In [16]:
class FeatureEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.ema1 = modules.EMA(1, 8)
        self.linear1 = nn.Linear(1 * 8, 11)
    def forward(self, x, h0):
        x, hn = self.ema1(x, h0)
        x = x.view(x.shape[0], -1)
        x = self.linear1(x)
        return x, hn
    
encoder = FeatureEncoder()
optmizer = optim.Adam(encoder.parameters(), lr=0.01)

In [18]:
epochs = 100
for epoch in range(epochs):
    hn = encoder.ema1.get_h0(torch.zeros(1))
    losses = []
    for batch, (batch_x, batch_y) in enumerate(loader):
        batch_y_pred, hn = encoder(batch_x, hn)

        batch_loss = loss_fn(batch_y_pred.squeeze(), batch_y)
#         print(f'\rBatch: {batch + 1} of {len(loader)}, loss: {batch_loss}', end='')
        losses.append(batch_loss)

    loss = torch.stack(losses).mean()
    loss.backward()
    optmizer.step()
    print(f'Epoch {epoch + 1} of {epochs}, loss: {loss}')
    

Epoch 1 of 100, loss: 2.272329807281494
Epoch 2 of 100, loss: 2.020160675048828
Epoch 3 of 100, loss: 1.789289951324463
Epoch 4 of 100, loss: 1.577691912651062
Epoch 5 of 100, loss: 1.38185453414917
Epoch 6 of 100, loss: 1.2004659175872803
Epoch 7 of 100, loss: 1.0351417064666748
Epoch 8 of 100, loss: 0.887071430683136
Epoch 9 of 100, loss: 0.7571144104003906
Epoch 10 of 100, loss: 0.6459916830062866
Epoch 11 of 100, loss: 0.5543099045753479
Epoch 12 of 100, loss: 0.48248961567878723
Epoch 13 of 100, loss: 0.4305798411369324
Epoch 14 of 100, loss: 0.39791157841682434
Epoch 15 of 100, loss: 0.38280463218688965
Epoch 16 of 100, loss: 0.38317012786865234
Epoch 17 of 100, loss: 0.3976632058620453
Epoch 18 of 100, loss: 0.4184553325176239
Epoch 19 of 100, loss: inf
Epoch 20 of 100, loss: nan
Epoch 21 of 100, loss: nan
Epoch 22 of 100, loss: nan
Epoch 23 of 100, loss: nan
Epoch 24 of 100, loss: nan
Epoch 25 of 100, loss: nan
Epoch 26 of 100, loss: nan
Epoch 27 of 100, loss: nan
Epoch 28 of 1